In [1]:
import pandas as pd
import re
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, classification_report, log_loss
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix


# Database connection setup
db_config = {
    'host': 'localhost',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'kaviyam123',
    'port': '5432'
}
connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

# Step 1: Load data from PostgreSQL
query = 'SELECT * FROM "corrected_merged_claim_data_EF";'
df = pd.read_sql(query, con=engine)

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

df_filtered = df[df['Policy Status'].isin(['Renewed', 'Not Renewed'])]

df_filtered['Status_Binary'] = df_filtered['Policy Status'].apply(lambda x: 1 if x == 'Not Renewed' else 0)

# Select relevant columns
selected_columns = ['Renewal Type', 'Product name ', 'Product name  2', 'biztype', 'age', 'Vehicle IDV', 
                    'Before GST Add-on GWP', 'Total OD Premium', 'Total TP Premium', 'gst', 'Total Premium Payable ', 'NCB % Previous Year', 
                    'Applicable Discount with NCB', 'Tie Up', 'Number of claims', 'Approved', 'Denied', 
                    'Status_Binary', 'Policy Tenure', 'Customer Tenure', 'New Customers', 'Claim Happaned/Not', 'Renewal Rate Status', 'WITHDRAWN']
df_selected = df_filtered[selected_columns]

# Convert numerical columns to float
numerical_columns = [ 'age', 'Vehicle IDV', 
                    'Before GST Add-on GWP', 'Total OD Premium', 'Total TP Premium', 'gst', 'Total Premium Payable ', 'NCB % Previous Year', 
                    'Applicable Discount with NCB', 'Number of claims', 'Approved', 'Denied', 
                    'Status_Binary', 'Policy Tenure', 'Customer Tenure', 'WITHDRAWN']
df_selected[numerical_columns] = df_selected[numerical_columns].apply(pd.to_numeric, errors='coerce')

# One-hot encode categorical columns
categorical_columns = ['Renewal Type', 'Product name ', 'Product name  2', 'biztype', 'Tie Up',
                       'New Customers', 'Claim Happaned/Not', 'Renewal Rate Status']
df_selected = pd.get_dummies(df_selected, columns=categorical_columns, drop_first=True)

# Impute missing values
imputer = SimpleImputer(strategy='constant', fill_value=0)
df_selected = pd.DataFrame(imputer.fit_transform(df_selected), columns=df_selected.columns)

# Convert to sparse matrix
X = csr_matrix(df_selected.drop(columns=['Status_Binary']).values)
y = df_selected['Status_Binary']

# Debugging dimensions
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

C:\Users\Admin\AppData\Local\Temp\ipykernel_13832\3154240862.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Status_Binary'] = df_filtered['Policy Status'].apply(lambda x: 1 if x == 'Not Renewed' else 0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13832\3154240862.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected[numerical_columns] = df_selected[numerical_columns].apply(pd.to_numeric, errors='coerce')


Shape of X: (1280454, 47)
Shape of y: (1280454,)


In [3]:
import numpy as np
import pandas as pd
import xgboost as xgb
import re
from collections import defaultdict

# Store feature dependencies
all_feature_dependencies = defaultdict(list)

# Prepare feature names
feature_names = df_selected.drop(columns=['Status_Binary']).columns.tolist()

# Train XGBoost model
print("\nTraining XGBoost Model - Single Run")

xgb_model = xgb.XGBClassifier(
    max_depth=5,                  
    learning_rate=0.05,            
    n_estimators=200,              
    subsample=0.8,                 
    colsample_bytree=0.8,         
    scale_pos_weight=len(y[y == 0]) / len(y[y == 1]),  
    gamma=0.1,                     
    random_state=42  # Fixed random state
) 

xgb_model.fit(X, y)

# Extract tree dump (all trees)
booster = xgb_model.get_booster()
tree_dumps = booster.get_dump()

formatted_trees = []

# Process each tree separately
for tree_index, tree_dump in enumerate(tree_dumps):
    formatted_tree = []
    indent = "    "  

    for line in tree_dump.split("\n"):
        if line.strip():  
            depth = line.count("\t")  
            formatted_line = line.replace("\t", indent)  

            # Replace feature indices (f0, f1, ...) with actual feature names
            for i, feature in enumerate(feature_names):
                formatted_line = formatted_line.replace(f"f{i}<", f"{feature} < ")
                formatted_line = formatted_line.replace(f"f{i}>", f"{feature} > ")

            # Fix incorrect thresholds (< 2.00001001 → < 0.5)
            formatted_line = re.sub(r"<\s*2\.00001001", "< 0.5", formatted_line)
            formatted_line = re.sub(r">=\s*2\.00001001", ">= 0.5", formatted_line)

            # Convert leaf values into class labels
            match = re.search(r"leaf=([-+]?\d*\.\d+)", formatted_line)
            if match:
                logit = float(match.group(1))
                probability = 1 / (1 + np.exp(-logit))  
                predicted_class = 1 if probability > 0.5 else 0  
                formatted_line = re.sub(r"leaf=[-+]?\d*\.\d+", f"class: {predicted_class}", formatted_line)

            formatted_line = formatted_line.replace("[", "|--- ").replace("]", "")
            formatted_tree.append(indent * depth + formatted_line)

    formatted_trees.append("\n".join(formatted_tree))

# Save formatted trees
with open("xgboost_tree_run.txt", "w") as file:
    file.write("\n\n".join(formatted_trees))

print("Decision trees saved as 'xgboost_tree_run.txt'")



Training XGBoost Model - Single Run
Decision trees saved as 'xgboost_tree_run.txt'


In [4]:
import re
from collections import defaultdict

# Load the tree rules from the file
file_path = "xgboost_tree_run.txt"

with open(file_path, "r") as file:
    tree_rules = file.readlines()

# Dictionary to store feature dependencies and their frequency
feature_threshold_counts = defaultdict(lambda: {"<": defaultdict(int), ">=": defaultdict(int)})

# Parse tree and extract rules leading to `class: 1`
def extract_non_renewal_conditions(tree_rules):
    current_path = []  # Stack to track decision path

    for line in tree_rules:
        line = line.strip()
        if not line:
            continue

        depth = line.count("|---")
        line_content = line.replace("|--- ", "").strip()

        # Handle decision nodes (conditions)
        match = re.search(r"([A-Za-z0-9_ ]+) (<|>) ([0-9\.]+) yes=(\d+),no=(\d+)", line_content)
        if match:
            feature, operator, threshold, yes_branch, no_branch = match.groups()
            threshold = float(threshold)

            # Identify `<` (left) or `>=` (right) branch
            current_path.append((depth, feature, operator, threshold, yes_branch, no_branch))

        # Handle leaf nodes (class 1)
        elif "class: 1" in line_content:
            # Find the decision path leading to `class: 1`
            for depth, feature, operator, threshold, yes_branch, no_branch in current_path:
                if str(yes_branch) in line:  # If class: 1 is in the left branch, `<` is important
                    feature_threshold_counts[feature]["<"][threshold] += 1
                elif str(no_branch) in line:  # If class: 1 is in the right branch, `>=` is important
                    feature_threshold_counts[feature][">="][threshold] += 1

    return feature_threshold_counts

# Extract dependencies and their frequency
feature_threshold_counts = extract_non_renewal_conditions(tree_rules)

# Save reasons to a file
output_file = "filtered_non_renewal_thresholds.txt"

with open(output_file, "w") as file:
    file.write("🔹 **Key Non-Renewal Conditions (Filtered for Most Important Condition Only)**:\n")
    file.write("=" * 90 + "\n\n")

    for feature, threshold_data in feature_threshold_counts.items():
        # Find the most frequent threshold for both `<` and `>=`
        most_common_less_than = max(threshold_data["<"], key=threshold_data["<"].get, default=None)
        most_common_greater_than = max(threshold_data[">="], key=threshold_data[">="].get, default=None)

        less_than_count = threshold_data["<"].get(most_common_less_than, 0)
        greater_than_count = threshold_data[">="].get(most_common_greater_than, 0)

        file.write(f"🔸 **{feature}** influences non-renewal due to:\n")

        # Only include the condition with a higher count
        if less_than_count > greater_than_count:
            file.write(f"   - {feature} < {most_common_less_than} (Most Frequent Condition)\n")
        elif greater_than_count > less_than_count:
            file.write(f"   - {feature} >= {most_common_greater_than} (Most Frequent Condition)\n")
        else:
            file.write(f"   - {feature} < {most_common_less_than} and >= {most_common_greater_than} (Equal Frequency, Review Needed)\n")

        file.write("-" * 90 + "\n")

print(f"\n✅ Updated non-renewal thresholds saved to '{output_file}'")


✅ Updated non-renewal thresholds saved to 'filtered_non_renewal_thresholds.txt'


In [5]:
import re
from collections import defaultdict

# Load the tree rules from the file
file_path = "xgboost_tree_run.txt"

with open(file_path, "r") as file:
    tree_rules = file.readlines()

# Dictionary to store feature dependencies and their frequency
feature_threshold_counts = defaultdict(lambda: {"<": defaultdict(int), ">=": defaultdict(int)})

# Parse tree and extract rules leading to `class: 1`
def extract_non_renewal_conditions(tree_rules):
    current_path = []  # Stack to track decision path

    for line in tree_rules:
        line = line.strip()
        if not line:
            continue

        depth = line.count("|---")
        line_content = line.replace("|--- ", "").strip()

        # Handle decision nodes (conditions)
        match = re.search(r"([A-Za-z0-9_ ]+) (<|>) ([0-9\.]+) yes=(\d+),no=(\d+)", line_content)
        if match:
            feature, operator, threshold, yes_branch, no_branch = match.groups()
            threshold = float(threshold)

            # Identify `<` (left) or `>=` (right) branch
            current_path.append((depth, feature, operator, threshold, yes_branch, no_branch))

        # Handle leaf nodes (class 1)
        elif "class: 1" in line_content:
            # Find the decision path leading to `class: 1`
            for depth, feature, operator, threshold, yes_branch, no_branch in current_path:
                if str(no_branch) in line:  # If class: 1 is in the right branch, `>=` is important
                    feature_threshold_counts[feature][">="][threshold] += 1
                elif str(yes_branch) in line:  # If class: 1 is in the left branch, `<` is important
                    feature_threshold_counts[feature]["<"][threshold] += 1

    return feature_threshold_counts

# Extract dependencies and their frequency
feature_threshold_counts = extract_non_renewal_conditions(tree_rules)

# Save reasons to a file
output_file = "filtered_non_renewal_thresholds.txt"

with open(output_file, "w") as file:
    file.write("🔹 **Key Non-Renewal Conditions (Filtered for Most Important Condition Only)**:\n")
    file.write("=" * 90 + "\n\n")

    for feature, threshold_data in feature_threshold_counts.items():
        # Find the most frequent `>=` threshold
        most_common_greater_than = max(threshold_data[">="], key=threshold_data[">="].get, default=None)
        greater_than_count = threshold_data[">="].get(most_common_greater_than, 0)

        # If no `>=` condition exists, fallback to `<`
        if most_common_greater_than is not None:
            file.write(f"🔸 **{feature}** influences non-renewal due to:\n")
            file.write(f"   - {feature} >= {most_common_greater_than} (Most Frequent Condition)\n")
            file.write("-" * 90 + "\n")
        else:
            # Only include `<` conditions if no `>=` exists
            most_common_less_than = max(threshold_data["<"], key=threshold_data["<"].get, default=None)
            less_than_count = threshold_data["<"].get(most_common_less_than, 0)

            if most_common_less_than is not None:
                file.write(f"🔸 **{feature}** influences non-renewal due to:\n")
                file.write(f"   - {feature} < {most_common_less_than} (No `>=` found, using `<` condition)\n")
                file.write("-" * 90 + "\n")

print(f"\n✅ Updated non-renewal thresholds saved to '{output_file}'")



✅ Updated non-renewal thresholds saved to 'filtered_non_renewal_thresholds.txt'


In [6]:
import re
from collections import defaultdict

# Load the tree rules from the file
file_path = "xgboost_tree_run.txt"

with open(file_path, "r") as file:
    tree_rules = file.readlines()

# Dictionary to store feature dependencies and their frequency
feature_threshold_counts = defaultdict(lambda: {"<": defaultdict(int), ">=": defaultdict(int)})

# Parse tree and extract rules leading to `class: 1`
def extract_non_renewal_conditions(tree_rules):
    current_path = []  # Stack to track decision path

    for line in tree_rules:
        line = line.strip()
        if not line:
            continue

        depth = line.count("|---")
        line_content = line.replace("|--- ", "").strip()

        # Handle decision nodes (conditions)
        match = re.search(r"([A-Za-z0-9_ ]+) (<|>) ([0-9\.]+) yes=(\d+),no=(\d+)", line_content)
        if match:
            feature, operator, threshold, yes_branch, no_branch = match.groups()
            threshold = float(threshold)

            # Identify `<` (left) or `>=` (right) branch
            current_path.append((depth, feature, operator, threshold, yes_branch, no_branch))

        # Handle leaf nodes (class 1)
        elif "class: 1" in line_content:
            # Find the decision path leading to `class: 1`
            for depth, feature, operator, threshold, yes_branch, no_branch in current_path:
                if str(yes_branch) in line:  # If class: 1 is in the left branch, `<` is important
                    feature_threshold_counts[feature]["<"][threshold] += 1
                elif str(no_branch) in line:  # If class: 1 is in the right branch, `>=` is important
                    feature_threshold_counts[feature][">="][threshold] += 1

    return feature_threshold_counts

# Extract dependencies and their frequency
feature_threshold_counts = extract_non_renewal_conditions(tree_rules)

# Save reasons to a file
output_file = "non_renewal_thresholds.txt"

with open(output_file, "w") as file:
    file.write("🔹 **Key Non-Renewal Conditions (Most Frequent Thresholds)**:\n")
    file.write("=" * 80 + "\n\n")

    for feature, threshold_data in feature_threshold_counts.items():
        most_common_less_than = max(threshold_data["<"], key=threshold_data["<"].get, default=None)
        most_common_greater_than = max(threshold_data[">="], key=threshold_data[">="].get, default=None)

        file.write(f"🔸 **{feature}** influences non-renewal due to:\n")
        if most_common_less_than is not None:
            file.write(f"   - {feature} < {most_common_less_than} (Most Common for Non-Renewal)\n")
        if most_common_greater_than is not None:
            file.write(f"   - {feature} >= {most_common_greater_than} (Most Common for Non-Renewal)\n")
        file.write("-" * 80 + "\n")

print(f"\n✅ Updated non-renewal thresholds saved to '{output_file}'")


✅ Updated non-renewal thresholds saved to 'non_renewal_thresholds.txt'


In [7]:
import re
from collections import defaultdict

# Load the tree rules from the file
file_path = "xgboost_tree_run.txt"

with open(file_path, "r") as file:
    tree_rules = file.readlines()

# Dictionary to store feature dependencies and their frequency
feature_threshold_counts = defaultdict(lambda: {"<": defaultdict(int), ">=": defaultdict(int)})

# Parse tree and extract rules leading to `class: 1`
def extract_non_renewal_conditions(tree_rules):
    current_path = []  # Stack to track decision path

    for line in tree_rules:
        line = line.strip()
        if not line:
            continue

        depth = line.count("|---")
        line_content = line.replace("|--- ", "").strip()

        # Handle decision nodes (conditions)
        match = re.search(r"([A-Za-z0-9_ ]+) (<|>) ([0-9\.]+) yes=(\d+),no=(\d+)", line_content)
        if match:
            feature, operator, threshold, yes_branch, no_branch = match.groups()
            threshold = float(threshold)

            # Identify `<` (left) or `>=` (right) branch
            current_path.append((depth, feature, operator, threshold, yes_branch, no_branch))

        # Handle leaf nodes (class 1)
        elif "class: 1" in line_content:
            # Find the decision path leading to `class: 1`
            for depth, feature, operator, threshold, yes_branch, no_branch in current_path:
                if str(yes_branch) in line:  # If class: 1 is in the left branch, `<` is important
                    feature_threshold_counts[feature]["<"][threshold] += 1
                elif str(no_branch) in line:  # If class: 1 is in the right branch, `>=` is important
                    feature_threshold_counts[feature][">="][threshold] += 1

    return feature_threshold_counts

# Extract dependencies and their frequency
feature_threshold_counts = extract_non_renewal_conditions(tree_rules)

# Save reasons to a file
output_file = "most_appropriate_non_renewal_thresholds.txt"

with open(output_file, "w") as file:
    file.write("🔹 **Key Non-Renewal Conditions (Most Appropriate Thresholds Selected)**:\n")
    file.write("=" * 90 + "\n\n")

    for feature, threshold_data in feature_threshold_counts.items():
        # Find the most frequent threshold for `<` and `>=`
        most_common_less_than = max(threshold_data["<"], key=threshold_data["<"].get, default=None)
        most_common_greater_than = max(threshold_data[">="], key=threshold_data[">="].get, default=None)

        less_than_count = threshold_data["<"].get(most_common_less_than, 0)
        greater_than_count = threshold_data[">="].get(most_common_greater_than, 0)

        file.write(f"🔸 **{feature}** influences non-renewal due to:\n")

        # Select the most frequent condition
        if less_than_count > greater_than_count:
            file.write(f"   - {feature} < {most_common_less_than} (Most Frequent for Non-Renewal, Count: {less_than_count})\n")
        elif greater_than_count > less_than_count:
            file.write(f"   - {feature} >= {most_common_greater_than} (Most Frequent for Non-Renewal, Count: {greater_than_count})\n")
        else:
            file.write(f"   - {feature} < {most_common_less_than} OR >= {most_common_greater_than} (Equal Frequency, Review Needed)\n")

        file.write("-" * 90 + "\n")

print(f"\n✅ Updated non-renewal thresholds saved to '{output_file}'")



✅ Updated non-renewal thresholds saved to 'most_appropriate_non_renewal_thresholds.txt'


In [8]:
import re
from collections import defaultdict

# Load the tree rules from the file
file_path = "xgboost_tree_run.txt"

with open(file_path, "r") as file:
    tree_rules = file.readlines()

# Dictionary to store feature dependencies and their frequency
feature_threshold_counts = defaultdict(lambda: {"<": defaultdict(int), ">=": defaultdict(int)})

# ================================
# 🚀 Set the selection criteria here
# ================================
SELECTION_MODE = "dominant"  # Options: "greater", "less", "dominant"
# "greater" → Keeps only `>=` conditions
# "less" → Keeps only `<` conditions
# "dominant" → Automatically selects the most frequent condition
# ================================

# Parse tree and extract rules leading to `class: 1`
def extract_non_renewal_conditions(tree_rules):
    current_path = []  # Stack to track decision path

    for line in tree_rules:
        line = line.strip()
        if not line:
            continue

        depth = line.count("|---")
        line_content = line.replace("|--- ", "").strip()

        # Handle decision nodes (conditions)
        match = re.search(r"([A-Za-z0-9_ ]+) (<|>) ([0-9\.]+) yes=(\d+),no=(\d+)", line_content)
        if match:
            feature, operator, threshold, yes_branch, no_branch = match.groups()
            threshold = float(threshold)

            # Identify `<` (left) or `>=` (right) branch
            current_path.append((depth, feature, operator, threshold, yes_branch, no_branch))

        # Handle leaf nodes (class 1)
        elif "class: 1" in line_content:
            # Find the decision path leading to `class: 1`
            for depth, feature, operator, threshold, yes_branch, no_branch in current_path:
                if str(yes_branch) in line:  # If class: 1 is in the left branch, `<` is important
                    feature_threshold_counts[feature]["<"][threshold] += 1
                elif str(no_branch) in line:  # If class: 1 is in the right branch, `>=` is important
                    feature_threshold_counts[feature][">="][threshold] += 1

    return feature_threshold_counts

# Extract dependencies and their frequency
feature_threshold_counts = extract_non_renewal_conditions(tree_rules)

# Save reasons to a file
output_file = f"non_renewal_thresholds_{SELECTION_MODE}.txt"

with open(output_file, "w") as file:
    file.write(f"🔹 **Key Non-Renewal Conditions ({SELECTION_MODE.upper()} Selection)**:\n")
    file.write("=" * 90 + "\n\n")

    for feature, threshold_data in feature_threshold_counts.items():
        # Find the most frequent threshold for `<` and `>=`
        most_common_less_than = max(threshold_data["<"], key=threshold_data["<"].get, default=None)
        most_common_greater_than = max(threshold_data[">="], key=threshold_data[">="].get, default=None)

        less_than_count = threshold_data["<"].get(most_common_less_than, 0)
        greater_than_count = threshold_data[">="].get(most_common_greater_than, 0)

        file.write(f"🔸 **{feature}** influences non-renewal due to:\n")

        # Conditionally write based on selection mode
        if SELECTION_MODE == "greater":
            if most_common_greater_than is not None:
                file.write(f"   - {feature} >= {most_common_greater_than} (Most Frequent, Count: {greater_than_count})\n")
        elif SELECTION_MODE == "less":
            if most_common_less_than is not None:
                file.write(f"   - {feature} < {most_common_less_than} (Most Frequent, Count: {less_than_count})\n")
        elif SELECTION_MODE == "dominant":
            if less_than_count > greater_than_count:
                file.write(f"   - {feature} < {most_common_less_than} (Most Frequent, Count: {less_than_count})\n")
            elif greater_than_count > less_than_count:
                file.write(f"   - {feature} >= {most_common_greater_than} (Most Frequent, Count: {greater_than_count})\n")
            else:
                file.write(f"   - {feature} < {most_common_less_than} OR >= {most_common_greater_than} (Equal Frequency, Review Needed)\n")

        file.write("-" * 90 + "\n")

print(f"\n✅ Updated non-renewal thresholds saved to '{output_file}'")


✅ Updated non-renewal thresholds saved to 'non_renewal_thresholds_dominant.txt'


In [9]:
import re
from collections import defaultdict

# Load the tree rules from the file
file_path = "xgboost_tree_run.txt"

with open(file_path, "r") as file:
    tree_rules = file.readlines()

# Dictionary to store feature dependencies and their frequency
feature_threshold_counts = defaultdict(lambda: {"<": defaultdict(int), ">=": defaultdict(int)})

# Parse tree and extract rules leading to `class: 1`
def extract_non_renewal_conditions(tree_rules):
    current_path = []  # Stack to track decision path

    for line in tree_rules:
        line = line.strip()
        if not line:
            continue

        depth = line.count("|---")
        line_content = line.replace("|--- ", "").strip()

        # Handle decision nodes (conditions)
        match = re.search(r"([A-Za-z0-9_ ]+) (<|>) ([0-9\.]+) yes=(\d+),no=(\d+)", line_content)
        if match:
            feature, operator, threshold, yes_branch, no_branch = match.groups()
            threshold = float(threshold)

            # Identify `<` (left) or `>=` (right) branch
            current_path.append((depth, feature, operator, threshold, yes_branch, no_branch))

        # Handle leaf nodes (class 1)
        elif "class: 1" in line_content:
            # Find the decision path leading to `class: 1`
            for depth, feature, operator, threshold, yes_branch, no_branch in current_path:
                if str(yes_branch) in line:  # If class: 1 is in the left branch, `<` is important
                    feature_threshold_counts[feature]["<"][threshold] += 1
                elif str(no_branch) in line:  # If class: 1 is in the right branch, `>=` is important
                    feature_threshold_counts[feature][">="][threshold] += 1

    return feature_threshold_counts

# Extract dependencies and their frequency
feature_threshold_counts = extract_non_renewal_conditions(tree_rules)

# Save reasons to a file
output_file = "non_renewal_thresholds_both_cases.txt"

with open(output_file, "w") as file:
    file.write("🔹 **Key Non-Renewal Conditions (Both `<` and `>=` Cases, Highlighting Most Dominant Condition)**:\n")
    file.write("=" * 100 + "\n\n")

    missing_greater_than = []  # Track features missing `>=` values

    for feature, threshold_data in feature_threshold_counts.items():
        # Find the most frequent threshold for `<` and `>=`
        most_common_less_than = max(threshold_data["<"], key=threshold_data["<"].get, default=None)
        most_common_greater_than = max(threshold_data[">="], key=threshold_data[">="].get, default=None)

        less_than_count = threshold_data["<"].get(most_common_less_than, 0)
        greater_than_count = threshold_data[">="].get(most_common_greater_than, 0)

        file.write(f"🔸 **{feature}** influences non-renewal due to:\n")

        # Display both `<` and `>=` conditions
        if most_common_less_than is not None:
            dominant_marker = "✅ " if less_than_count > greater_than_count else ""
            file.write(f"   {dominant_marker}- {feature} < {most_common_less_than} (Count: {less_than_count})\n")

        if most_common_greater_than is not None:
            dominant_marker = "✅ " if greater_than_count > less_than_count else ""
            file.write(f"   {dominant_marker}- {feature} >= {most_common_greater_than} (Count: {greater_than_count})\n")

        if most_common_greater_than is None:
            missing_greater_than.append(feature)  # Log missing `>=` values

        file.write("-" * 100 + "\n")

    # Log missing `>=` values
    if missing_greater_than:
        file.write("\n🔴 **WARNING: The following features had no `>=` condition occurrences:**\n")
        file.write(", ".join(missing_greater_than) + "\n")
        file.write("=" * 100 + "\n")

print(f"\n✅ Updated non-renewal thresholds saved to '{output_file}'")
if missing_greater_than:
    print(f"\n⚠️ WARNING: No `>=` conditions found for these features: {', '.join(missing_greater_than)}")



✅ Updated non-renewal thresholds saved to 'non_renewal_thresholds_both_cases.txt'
